In [7]:
! pip install xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb


In [8]:
# Load data from SQLite database
query = 'SELECT * FROM rose_wine'
conn = sqlite3.connect('wine_database.db')
df = pd.read_sql_query(query, conn)
conn.close()

columns_to_decode = ['Name', 'Region', 'Winery']
for col in columns_to_decode:
    df[col] = df[col].str.encode('iso-8859-1').str.decode('utf-8')
   
df = df.drop(columns=['Name'])

# Separate categorical and numerical features
categorical_cols = ['Country', 'Region', 'Winery']
numerical_cols = ['Price', 'Year']

X = df[categorical_cols + numerical_cols]
y = df['Rating']
X

,Country,Region,Winery,Price,Year
0,France,Méditerranée,Château Camparnaud,10.11,2019
1,Portugal,Douro,Niepoort,12.81,2019
2,Italy,Abruzzo,Farnese,5.60,2019
3,Israel,Galilee,Hermon,12.14,2017
4,France,Languedoc,Château La Sauvageonne,68.95,2018
...,...,...,...,...,...
389,France,Coteaux d'Aix-en-Provence,AIX,13.90,2019
390,Italy,Lombardia,Cà dei Frati,11.20,2019
391,France,Languedoc,Gérard Bertrand,9.36,2019
392,France,Côtes de Provence,Minuty,12.85,2019


In [9]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Combine training and test data to fit the encoder
X_combined = pd.concat([X_train, X_test], ignore_index=True)

# Create transformers for preprocessing
cat_transformer = OneHotEncoder(handle_unknown='ignore')
num_transformer = StandardScaler()

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, categorical_cols),
        ('num', num_transformer, numerical_cols)
    ]
)

# Fit and transform the combined dataset
X_combined_preprocessed = preprocessor.fit_transform(X_combined)

# Split the preprocessed data back into training and test datasets
X_train_preprocessed = X_combined_preprocessed[:len(X_train)]
X_test_preprocessed = X_combined_preprocessed[len(X_train):]

# Create XGBoost model
xgb_model = xgb.XGBRegressor()

# Fit the grid search to the preprocessed training data
xgb_model.fit(X_train_preprocessed, y_train)

# Make predictions on the preprocessed test data
predictions = xgb_model.predict(X_test_preprocessed)

# Calculate R-squared and Mean Squared Error
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print(f"R-squared: {r2:.4f}")
print(f"Mean Squared Error: {mse:.4f}")

R-squared: 0.4024
Mean Squared Error: 0.0471
